In [3]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as ppl

In [4]:
raw_data = pd.read_csv("data/train.csv")

Item_Identifier ---- Unique product ID

Item_Weight ---- Weight of product

Item_Fat_Content ---- Whether the product is low fat or not

Item_Visibility ---- The % of the total display area of all products in a store allocated to the particular product

Item_Type ---- The category to which the product belongs

Item_MRP ---- Maximum Retail Price (list price) of the product

Outlet_Identifier ---- Unique store ID

Outlet_Establishment_Year ---- The year in which the store was established

Outlet_Size ---- The size of the store in terms of ground area covered

Outlet_Location_Type ---- The type of city in which the store is located

*Outlet_Type ---- Whether the outlet is just a grocery store or some sort of supermarket

Item_Outlet_Sales ---- sales of the product in t particular store. This is the outcome variable to be predicted.


### Data Exploring

In [5]:
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                7060 non-null   float64
 2   Item_Fat_Content           8523 non-null   object 
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Identifier          8523 non-null   object 
 7   Outlet_Establishment_Year  8523 non-null   int64  
 8   Outlet_Size                6113 non-null   object 
 9   Outlet_Location_Type       8523 non-null   object 
 10  Outlet_Type                8523 non-null   object 
 11  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 799.2+ KB


#### Dealing with missing

In [6]:
cnt = raw_data.shape[0]
missing = pd.Series()
for col in raw_data:
    n = raw_data[col].loc[raw_data[col].notna() == False].size
    if n != cnt:
        missing[col] = n
missing.loc[missing != 0]

Item_Weight    1463
Outlet_Size    2410
dtype: int64

In [7]:
raw_data.groupby(['Outlet_Identifier']).count()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
Outlet_Identifier,,,,,,,,,,,
OUT010,555,555,555,555,555,555,555,0,555,555,555
OUT013,932,932,932,932,932,932,932,932,932,932,932
OUT017,926,926,926,926,926,926,926,0,926,926,926
OUT018,928,928,928,928,928,928,928,928,928,928,928
OUT019,528,0,528,528,528,528,528,528,528,528,528
OUT027,935,0,935,935,935,935,935,935,935,935,935
OUT035,930,930,930,930,930,930,930,930,930,930,930
OUT045,929,929,929,929,929,929,929,0,929,929,929
OUT046,930,930,930,930,930,930,930,930,930,930,930


##### Outlet size

In [8]:
missing = raw_data.loc[raw_data['Outlet_Size'].notna() == False]
present = raw_data.loc[raw_data['Outlet_Size'].notna() == True]

In [11]:
missing.groupby(['Outlet_Identifier']).count()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
Outlet_Identifier,,,,,,,,,,,
OUT010,555,555,555,555,555,555,555,0,555,555,555
OUT017,926,926,926,926,926,926,926,0,926,926,926
OUT045,929,929,929,929,929,929,929,0,929,929,929


Outlet size is completely missing on every occurrence

##### Item weight

In [12]:
missing = raw_data.loc[raw_data['Item_Weight'].notna() == False]
present = raw_data.loc[raw_data['Item_Weight'].notna() == True]

In [19]:
missing['Outlet_Identifier'].unique()

array(['OUT027', 'OUT019'], dtype=object)

Data missing from OUT027 and OUT019

Searching for identical goods in other outlets

In [88]:
missing_all_info = raw_data[raw_data['Item_Identifier'].isin(missing['Item_Identifier'])]
missing_all_info

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
6,FDO10,13.65,Regular,0.012741,Snack Foods,57.6588,OUT013,1987,High,Tier 3,Supermarket Type1,343.5528
...,...,...,...,...,...,...,...,...,...,...,...,...
8513,FDH31,12.00,Regular,0.020407,Meat,99.9042,OUT035,2004,Small,Tier 2,Supermarket Type1,595.2252
8514,FDA01,15.00,Regular,0.054489,Canned,57.5904,OUT045,2002,NaN,Tier 2,Supermarket Type1,468.7232
8516,NCJ19,18.60,Low Fat,0.118661,Others,58.7588,OUT018,2009,Medium,Tier 3,Supermarket Type2,858.8820
8520,NCJ29,10.60,Low Fat,0.035186,Health and Hygiene,85.1224,OUT035,2004,Small,Tier 2,Supermarket Type1,1193.1136


Finding goods with values to copy and with no clues

In [132]:
missings_found_value = missing_all_info.groupby('Item_Identifier').count().query('Item_Weight != 0')
missings_found_value = missings_found_value[['Item_Weight', 'Outlet_Identifier']]
missings_found_value

,Item_Weight,Outlet_Identifier
Item_Identifier,,
DRA24,5,7
DRA59,6,8
DRB01,2,3
DRB25,5,6
DRB48,6,7
...,...,...
NCZ17,5,6
NCZ18,6,7
NCZ30,6,7


Filling with found data

In [150]:
res = pd.DataFrame()
for item_id in missings_found_value.iloc[:, 0].index:
    res[item_id]

DRA24
DRA59
DRB01
DRB25
DRB48
DRC01
DRC25
DRC36
DRC49
DRD12
DRD13
DRD15
DRD24
DRD25
DRD27
DRD60
DRE12
DRE13
DRE15
DRE25
DRE37
DRE48
DRE49
DRE60
DRF01
DRF03
DRF15
DRF23
DRF25
DRF27
DRF36
DRF37
DRF49
DRG01
DRG03
DRG13
DRG23
DRG27
DRG36
DRG37
DRG48
DRG49
DRH01
DRH03
DRH11
DRH13
DRH15
DRH23
DRH25
DRH36
DRH37
DRH39
DRH49
DRH51
DRI01
DRI03
DRI11
DRI23
DRI25
DRI37
DRI39
DRI51
DRI59
DRJ01
DRJ23
DRJ24
DRJ37
DRJ39
DRJ47
DRJ59
DRK11
DRK12
DRK13
DRK23
DRK35
DRK37
DRK39
DRK49
DRK59
DRL01
DRL11
DRL35
DRL37
DRL47
DRL59
DRL60
DRM23
DRM35
DRM37
DRM47
DRM48
DRM59
DRN36
DRN47
DRN59
DRO59
DRP35
DRQ35
DRY23
DRZ11
DRZ24
FDA01
FDA02
FDA03
FDA04
FDA07
FDA08
FDA10
FDA11
FDA13
FDA14
FDA15
FDA16
FDA19
FDA20
FDA22
FDA23
FDA26
FDA28
FDA32
FDA34
FDA35
FDA36
FDA38
FDA39
FDA43
FDA44
FDA45
FDA46
FDA47
FDA49
FDA50
FDA51
FDA52
FDA55
FDA56
FDA57
FDB02
FDB04
FDB05
FDB08
FDB09
FDB14
FDB17
FDB21
FDB22
FDB23
FDB26
FDB27
FDB32
FDB33
FDB34
FDB35
FDB36
FDB37
FDB38
FDB39
FDB40
FDB41
FDB44
FDB45
FDB49
FDB52
FDB53
FDB56
FDB57
FDB5

Filling the no-clues ones

In [174]:
missing_no_info = missing_all_info.groupby('Item_Identifier').nunique().query('Item_Weight == 0').index
missing_no_info

Index(['FDE52', 'FDK57', 'FDN52', 'FDQ60'], dtype='object', name='Item_Identifier')

In [175]:
raw_data[raw_data['Item_Identifier'].isin(missing_no_info)]

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
927,FDN52,NaN,Regular,0.130933,Frozen Foods,86.9198,OUT027,1985,Medium,Tier 3,Supermarket Type3,1569.9564
1922,FDK57,NaN,Low Fat,0.079904,Snack Foods,120.0440,OUT027,1985,Medium,Tier 3,Supermarket Type3,4434.2280
4187,FDE52,NaN,Regular,0.029742,Dairy,88.9514,OUT027,1985,Medium,Tier 3,Supermarket Type3,3453.5046
5022,FDQ60,NaN,Regular,0.191501,Baking Goods,121.2098,OUT019,1985,Small,Tier 1,Grocery Store,120.5098


In [189]:
raw_data.loc[:, ['Item_Weight', 'Item_Type']].groupby('Item_Type').median()

,Item_Weight
Item_Type,
Baking Goods,11.650
Breads,10.600
Breakfast,10.695
Canned,12.150
Dairy,13.350
Frozen Foods,12.850
Fruits and Vegetables,13.100
Hard Drinks,10.100
Health and Hygiene,12.150


In [128]:
raw_data.query('Item_Identifier == "FDQ60"')

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
5022,FDQ60,NaN,Regular,0.191501,Baking Goods,121.2098,OUT019,1985,Small,Tier 1,Grocery Store,120.5098


#### Data likehood

In [ ]:
cols = pd.Series()
for col in raw_data.columns:
    cols[col] = raw_data[col].nunique()
cols.sort_values(ascending=False)

Item_Visibility              7880
Item_MRP                     5938
Item_Outlet_Sales            3493
Item_Identifier              1559
Item_Weight                   415
Item_Type                      16
Outlet_Identifier              10
Outlet_Establishment_Year       9
Item_Fat_Content                5
Outlet_Type                     4
Outlet_Size                     3
Outlet_Location_Type            3
dtype: int64

In [ ]:
data = raw_data[['Outlet_Identifier', 'Item_Identifier', 'Item_Outlet_Sales', 'Item_Visibility', 'Item_MRP', 'Item_Weight', 'Item_Type', 'Item_Fat_Content', 'Outlet_Establishment_Year', 'Outlet_Type', 'Outlet_Size', 'Outlet_Location_Type']]

In [ ]:
goods = data[['Item_Identifier', 'Outlet_Identifier', 'Item_Outlet_Sales', 'Item_Visibility', 'Item_MRP', 'Item_Weight', 'Item_Type', 'Item_Fat_Content']].sort_values('Item_Identifier')

In [ ]:
outlets = data[['Outlet_Identifier', 'Outlet_Establishment_Year', 'Outlet_Type', 'Outlet_Size', 'Outlet_Location_Type']].groupby("Outlet_Identifier").first()
outlets

,Outlet_Establishment_Year,Outlet_Type,Outlet_Size,Outlet_Location_Type
Outlet_Identifier,,,,
OUT010,1998,Grocery Store,None,Tier 3
OUT013,1987,Supermarket Type1,High,Tier 3
OUT017,2007,Supermarket Type1,None,Tier 2
OUT018,2009,Supermarket Type2,Medium,Tier 3
OUT019,1985,Grocery Store,Small,Tier 1
OUT027,1985,Supermarket Type3,Medium,Tier 3
OUT035,2004,Supermarket Type1,Small,Tier 2
OUT045,2002,Supermarket Type1,None,Tier 2
OUT046,1997,Supermarket Type1,Small,Tier 1


In [ ]:
def get_outlet_info(key):
    display(goods.loc[goods['Outlet_Identifier'] == key].drop(['Outlet_Identifier'], axis=1).sort_index())

In [ ]:
get_outlet_info('OUT049')

,Item_Identifier,Item_Outlet_Sales,Item_Visibility,Item_MRP,Item_Weight,Item_Type,Item_Fat_Content
0,FDA15,3735.1380,0.016047,249.8092,9.300,Dairy,Low Fat
2,FDN15,2097.2700,0.016760,141.6180,17.500,Meat,Low Fat
10,FDY07,1516.0266,0.000000,45.5402,11.800,Fruits and Vegetables,Low Fat
12,FDX32,1589.2646,0.100014,145.4786,15.100,Fruits and Vegetables,Regular
17,FDP49,718.3982,0.069196,54.3614,9.000,Breakfast,Regular
...,...,...,...,...,...,...,...
8451,FDK21,3254.4304,0.010028,249.6408,7.905,Snack Foods,Low Fat
8467,FDV31,3881.6140,0.000000,175.2370,9.800,Fruits and Vegetables,LF
8470,FDW27,1551.3140,0.151088,155.1314,5.860,Meat,Regular
8475,NCS17,378.1744,0.080627,92.5436,18.600,Health and Hygiene,Low Fat
